  Using cached pandas-1.5.3-cp310-cp310-macosx_11_0_arm64.whl (10.9 MB)
  Using cached pytz-2022.7.1-py2.py3-none-any.whl (499 kB)


Step-1 : Create your annotations with the help of an external tool

https://tecoholic.github.io/ner-annotator/

In [1]:
import pandas as pd
import json
import os
with open('annotations.json', 'r') as f:
    data = json.load(f)

In [2]:
data


{'classes': ['AGENDA'],
 'annotations': [["Schedule a meeting next week between John, Mary, and Jane to discuss the new product launch. We'll need 1 hour.",
   {'entities': [[73, 92, 'AGENDA']]}],
  ['Can we meet on Friday to talk about the sales figures from last quarter? I need to prepare a presentation.',
   {'entities': [[40, 53, 'AGENDA']]}],
  ["Let's get together tomorrow to brainstorm ideas for the upcoming marketing campaign.",
   {'entities': [[31, 41, 'AGENDA']]}],
  ["chedule a meeting next Wednesday with Alex and Jim to discuss the new marketing campaign. We'll need 2 hours",
   {'entities': [[66, 89, 'AGENDA']]}],
  ['We need to discuss the budget for the next quarter during our meeting tomorrow.',
   {'entities': [[23, 29, 'AGENDA']]}],
  ['We need to talk about the sales figures from last month during our meeting on Friday.',
   {'entities': [[26, 39, 'AGENDA']]}],
  ['Please schedule a meeting for next Tuesday to discuss the new project proposal.',
   {'entities': [[58

In [3]:
entity_name = "AGENDA"

train_data = data['annotations']
train_data = [tuple(i) for i in train_data]

In [4]:
train_data
for i in train_data:
    if i[1]['entities'] == []:
        i[1]['entities'] = (0, 0, entity_name)
    else:
        i[1]['entities'][0] = tuple(i[1]['entities'][0])

In [5]:
train_data

[("Schedule a meeting next week between John, Mary, and Jane to discuss the new product launch. We'll need 1 hour.",
  {'entities': [(73, 92, 'AGENDA')]}),
 ('Can we meet on Friday to talk about the sales figures from last quarter? I need to prepare a presentation.',
  {'entities': [(40, 53, 'AGENDA')]}),
 ("Let's get together tomorrow to brainstorm ideas for the upcoming marketing campaign.",
  {'entities': [(31, 41, 'AGENDA')]}),
 ("chedule a meeting next Wednesday with Alex and Jim to discuss the new marketing campaign. We'll need 2 hours",
  {'entities': [(66, 89, 'AGENDA')]}),
 ('We need to discuss the budget for the next quarter during our meeting tomorrow.',
  {'entities': [(23, 29, 'AGENDA')]}),
 ('We need to talk about the sales figures from last month during our meeting on Friday.',
  {'entities': [(26, 39, 'AGENDA')]}),
 ('Please schedule a meeting for next Tuesday to discuss the new project proposal.',
  {'entities': [(58, 78, 'AGENDA')]}),
 ("Let's have a meeting to brains

In [5]:
import pandas as pd
import os
from tqdm import tqdm
import spacy
from spacy.tokens import DocBin

#nlp = spacy.blank("en") # load a new spacy model
nlp = spacy.load("en_core_web_sm") # load other spacy model

db = DocBin() # create a DocBin object

for text, annot in tqdm(train_data): # data in previous format
    doc = nlp.make_doc(text) # create doc object from text
    ents = []
    for start, end, label in annot["entities"]: # add character indexes
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is None:
            print("Skipping entity")
        else:
            ents.append(span)
    doc.ents = ents # label the text with the ents
    db.add(doc)

db.to_disk("./train.spacy") # save the docbin object

100%|██████████| 8/8 [00:00<00:00, 3181.12it/s]


In [6]:
!python -m spacy train config.cfg --output ./output --paths.train ./train.spacy --paths.dev ./train.spacy 

✔ Created output directory: output
ℹ Saving to output directory: output
ℹ Using CPU

=========================== Initializing pipeline ===========================
[2023-03-12 20:08:58,794] [INFO] Set up nlp object from config
[2023-03-12 20:08:58,800] [INFO] Pipeline: ['tok2vec', 'ner']
[2023-03-12 20:08:58,802] [INFO] Created vocabulary
[2023-03-12 20:08:59,058] [WARNING] [W112] The model specified to use for initial vectors (en_core_web_sm) has no vectors. This is almost certainly a mistake.
[2023-03-12 20:08:59,059] [INFO] Added vectors: en_core_web_sm
[2023-03-12 20:08:59,059] [INFO] Finished initializing nlp object
[2023-03-12 20:08:59,143] [INFO] Initialized pipeline components: ['tok2vec', 'ner']
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  -----

In [10]:
nlp1 = spacy.load("./output/model-best") #load the best model
doc = nlp1("Schedule a meeting next Wednesday with Alex and Jim to discuss the product launch. We'll need 2 hours.") # input sample text

spacy.displacy.render(doc, style="ent", jupyter=True)